<h1>PDF Scrape</h1>

In [1]:
# !pip install PyPDF2
# !pip install textract
# pip install tabula-py
# !pip install nltk
# !pip install camelot-py[all]
## for camelot need to install GPL GhostScript(Google it) for it to run,require admin permission to install camelot

In [5]:
import PyPDF2 as p2
import os
import camelot
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time, sys
import shutil
import re

In [6]:
# make folder to save csv files
f not os.path.isdir('uncleaned csv'):
    os.makedirs('uncleaned csv')

#### Extract PDF and save in csv
The function below will try to extract the table of the pdf files base on the keyword, if able to find the page page and extracted the tables save it in the directory,it will return True else return False

In [6]:
def save_shareholder_to_csv(pdf_file_path , company_name):
    if not os.path.isdir('shareholder'):
        os.makedirs('shareholder')
        
    try:
#       directory to save the extracted tables
        directory = "./uncleaned csv/"
        print(pdf_file_path+"\n")
        PDFFile = open(pdf_file_path, "rb")
        pdfread = p2.PdfFileReader(PDFFile)

#       list of word to search
        search_word_list1 = ["TWENTY" ,"LARGEST" ,"SHAREHOLDERS"]
        
        search_word_list2 = ["TOP" , "20" , "SHAREHOLDERS"]
        search_word_list3 = ["TOP" , "TWENTY", "SHAREHOLDERS"]
        search_word_list4 = ["20" , "LARGEST", "SHAREHOLDERS"]
        search_word_list5 = ["TWENTY", "LARGEST", "UNITHOLDERS"]
        search_word_list6 = ["20", "LARGEST", "UNITHOLDERS"]
        search_word_list7 = ["TOP","TWENTY", "UNITHOLDERS"]
        search_word_list8 = ["TOP","20","UNITHOLDERS"]
        search_word_list9 = ["","MAJOR","SHAREHOLDERS"]
        
        
        whole_list = []
        whole_list.extend((search_word_list1,search_word_list2 ,search_word_list3,
                           search_word_list4,search_word_list5,search_word_list6,
                           search_word_list7,search_word_list8,search_word_list9
                          ))
        print(whole_list)
        for partial_list in whole_list:

            search_word1 = partial_list[0]
            search_word2 = partial_list[1]
            search_word3 = partial_list[2]


            number_of_pages = pdfread.getNumPages()
            print("\n..............\n")
#             loop through the whole page in the pdf
            for i in range(number_of_pages):
                sys.stdout.write("\rpage : %d /%d" % (i,number_of_pages) )
                sys.stdout.flush()
                search_word_count_1 = 0
                search_word_count_2 = 0
                search_word_count_3 = 0

                if pdfread.getPage(i).getContents() == None:
                    i = i + 1
                    continue
                pageinfo = pdfread.getPage(i)
                text = pageinfo.extractText()
                text = text.upper()
                text = text.replace("\n", "")
                search_text = text.upper().split()

                # if text not empty set text else continue next page
                if text != "":
                    text = text

                else:
                    continue
                    
                #The word_tokenize() function will break our text phrases into #individual words
                tokens = word_tokenize(text)
                #we'll create a new list which contains punctuation we wish to clean
                punctuations = ['(',')',';',':','[',']',',']
                #We initialize the stopwords variable which is a list of words like #"The", "I", "and", etc. that don't hold much value as keywords
                stop_words = stopwords.words('english')
                #We create a list comprehension which only returns a list of words #that are NOT IN stop_words and NOT IN punctuations.
                keywords = [word for word in tokens if not word in stop_words and not word in punctuations]
                  
#                 if word matches the search word add counter
                for word in keywords:
                        if search_word1 in word:
                            search_word_count_1 += 1

                        if search_word2 in word:
                            search_word_count_2 += 1

                        if search_word3 in word:
                            search_word_count_3 += 1

#  if the 3 search word have at least 1 count we try to extract the table

                if(search_word_count_1>0 and search_word_count_2>0 and search_word_count_3>0):
                    search_word_count_total = search_word_count_1  + search_word_count_2 + search_word_count_3 
#                     display(keywords)
                    page = i + 1
                    print("page " + str(page))
                    print("The words was found {} times in total".format(search_word_count_total))
                    df = camelot.read_pdf(pdf_file_path,pages=str(page),flavor="stream",edge_tol=500)
        #             camelot.plot(df[0], kind='contour')
                    for n in df:
                        current_df = n.df
                        for i in current_df:
                            current_df[i] = current_df[i].str.upper()
                        table_dataframe = current_df
                        index = None
                        searchfor = [search_word2,search_word3]
                        
#                         check for the correct table
                        if(current_df[0].str.contains('|'.join(searchfor)).any() | current_df[1].str.contains('|'.join(searchfor)).any()):
                            print("table found")
                        else:
                            display(current_df)
                            continue
                        col_index=""
                        for column in current_df:
                            print("search col")
                            if current_df[column].str.contains("NAME").any():
                                print("have name")
                                col_index = current_df.index[current_df[column].str.contains("NAME")]

                            if len(col_index) != 0:
                                index = col_index[0].item()
                                print("index : " + str(index)) 
                        if index == None:
                            print("unable to find index")
                            continue                    
        #                 if(current_df[0].str.contains("TWENTY LARGEST SHAREHOLDERS").any()):
        #                     table_dataframe = current_df
        #                      display(table_dataframe)
        #                     index = table_dataframe.index[table_dataframe[0].str.contains("TWENTY LARGEST SHAREHOLDERS")].item()
        #                     print("index " +str(index))
        
            #         do some form of cleaning
                        print("start cleaning")
                        while index != 0:

                            table_dataframe.drop([0],inplace=True)
                            table_dataframe = table_dataframe.reset_index(drop=True)
                            index = index - 1

                        table_dataframe.columns = table_dataframe.iloc[0,:]
                        table_dataframe.drop(table_dataframe.index[0] , inplace= True)
                        table_dataframe.reset_index(inplace = True , drop=True) 
                        display(table_dataframe)
                        path = directory+company_name+" shares"+".csv"
                        print(path)
                        table_dataframe.to_csv(path, index = False)
                        print("True")
                        PDFFile.close()
                        if PDFFile.closed:
                            print('file is closed')
                        return True
                    
        text = "UNABLE TO FIND"
        print(text)
        PDFFile.close()
        if PDFFile.closed:
            print('file is closed')
        return False

    except Exception as e: 
        print(e)
        PDFFile.close()
        if PDFFile.closed:
            print('file is closed')
        return False

<div class="alert alert-block alert-warning">
<b>Warning:</b> Table extracted might not be 100% correct ,please monitor the output.
</div>

#### Loop through all the pdf files in the directory
Get company name and path and fit into the function,append all company names that failed the extraction in a list

In [7]:
directory = './pdf_files/'
# moveto = "./pdf_files/"
unable_to_find_list = []
# lo
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        company_name = filename.replace("_2018_Annual_Report.pdf","")
        company_name = company_name.strip()
        print(company_name)
#         observe output the extracted table might not be correct
        able_to_find = save_shareholder_to_csv(directory+filename,company_name)
        time.sleep(3)
        if able_to_find == False:
#             shutil.move(directory+filename,directory+"DONE/"+filename)
#         else:
            unable_to_find_list.append(company_name)
    else:
        continue
print(unable_to_find_list)


BLACKGOLD NATURAL RESOURCES LIMITED
./missed_out_pdf_files/BLACKGOLD NATURAL RESOURCES LIMITED_2018_Annual_Report.pdf

[['TWENTY', 'LARGEST', 'SHAREHOLDERS'], ['TOP', '20', 'SHAREHOLDERS'], ['TOP', 'TWENTY', 'SHAREHOLDERS'], ['20', 'LARGEST', 'SHAREHOLDERS'], ['TWENTY', 'LARGEST', 'UNITHOLDERS'], ['20', 'LARGEST', 'UNITHOLDERS'], ['TOP', 'TWENTY', 'UNITHOLDERS'], ['TOP', '20', 'UNITHOLDERS'], ['', 'MAJOR', 'SHAREHOLDERS']]

..............

page : 107 /108
..............

page : 28 /108page 29
The words was found 18 times in total


,0,1,2,3,4,5
0,NAME OF TOP 5 MANAGEMENT,,,,,
1,PERSONNEL (WHO ARE NOT,BASE,DIRECTORS’,,PROVIDENT FUND,
2,DIRECTORS OR CEO),SALARY,FEE,BONUS,CONTRIBUTION,TOTAL
3,"BELOW S$250,000",(%),(%),(%),(%),(%)
4,SUHERMAN BUDIONO,100,–,–,–,100
5,JEREMY NG7,85,–,7,8,100
6,RUDY HERLAMBANG,100,–,–,–,100
7,LILI HARYANTO,100,–,–,–,100
8,NATALIA HERMANTO,92,–,8,–,100
9,NOTES:–,,,,,


table found
search col
search col
unable to find index
page : 96 /108page 97
The words was found 11 times in total
table found
search col
search col
search col
search col
search col
search col
unable to find index
table found
search col
search col
search col
unable to find index
table found
search col
search col
have name
index : 2
search col
index : 2
search col
index : 2
search col
index : 2
start cleaning


,NO.,NAME OF SHAREHOLDERS,,NO. OF SHARES,%
0,1,UOB KAY HIAN PTE LTD,,"434,018,891",43.24
1,2,BNP PARIBAS NOMINEES SINGAPORE PTE LTD,,"89,662,556",8.93
2,3,DBS NOMINEES PTE LTD,,"50,653,785",5.05
3,4,OCBC SECURITIES PRIVATE LTD,,"29,881,109",2.98
4,5,YAO HSIAO TUNG,,"21,800,000",2.17
5,6,PHILLIP SECURITIES PTE LTD,,"19,047,900",1.90
6,7,RAFFLES NOMINEES (PTE) LTD,,"15,774,800",1.57
7,8,UNITED OVERSEAS BANK NOMINEES PTE LTD,,"10,857,500",1.08
8,9,CITIBANK NOMINEES SINGAPORE PTE LTD,,"10,320,500",1.03
9,10,GOH GUAN SIONG (WU YUANXIANG),,"9,280,000",0.92


./missed_out_pdf_files/uncleaned csv/BLACKGOLD NATURAL RESOURCES LIMITED shares.csv
True
file is closed
DELFI LIMITED
./missed_out_pdf_files/DELFI LIMITED_2018_Annual_Report.pdf

[['TWENTY', 'LARGEST', 'SHAREHOLDERS'], ['TOP', '20', 'SHAREHOLDERS'], ['TOP', 'TWENTY', 'SHAREHOLDERS'], ['20', 'LARGEST', 'SHAREHOLDERS'], ['TWENTY', 'LARGEST', 'UNITHOLDERS'], ['20', 'LARGEST', 'UNITHOLDERS'], ['TOP', 'TWENTY', 'UNITHOLDERS'], ['TOP', '20', 'UNITHOLDERS'], ['', 'MAJOR', 'SHAREHOLDERS']]

..............

page : 191 /228

...

'9. INTEREST-BEARING BORROWINGS (CONT™D)THE NOMINAL INTEREST RATE FOR THE S$ ˚OATING RATE LOANS IS DETERMINED BY A MARGIN PLUS SOR PER ANNUM.AS AT THE REPORTING DATE, THE GROUP HAS IN PLACE UNSECURED BORROWINGS COMPRISING:(A) TERM LOANS AND REVOLVING CREDIT FACILITIES(I) 4-YEAR $150 MILLION LOAN FACILITY FROM CIMB MATURING IN JUNE 2019 (FITLF1FL) CONSISTING OF: Ł FACILITY A: $100 MILLION TERM LOAN FACILITY AT A ˜XED RATE OF 3.6% PER ANNUM FOR 3.5 YEARS FROM THE DATE OF LOAN DRAWN DOWN; AND Ł FACILITY B: $50 MILLION REVOLVING CREDIT FACILITY AT AN INTEREST RATE OF MARGIN PLUS SWAP OFFER RATE. (II) 4.75-YEAR $200 MILLION LOAN FACILITY FROM HSBC MATURING IN JUNE 2021 (FITLF2FL) CONSISTING OF: Ł FACILITY A: $25 MILLION TERM LOAN FACILITY AT AN INTEREST RATE OF MARGIN PLUS SWAP OFFER RATE FOR 4.75 YEARS FROM THE DATE OF LOAN DRAWN DOWN; AND Ł FACILITY B: $175 MILLION REVOLVING CREDIT FACILITY AT AN INTEREST RATE OF MARGIN PLUS SWAP OFFER RATE. (III) LOAN FACILITY OF $700 MILLION FROM A SYND

['9.',
 'INTEREST-BEARING',
 'BORROWINGS',
 '(CONT™D)THE',
 'NOMINAL',
 'INTEREST',
 'RATE',
 'FOR',
 'THE',
 'S$',
 '˚OATING',
 'RATE',
 'LOANS',
 'IS',
 'DETERMINED',
 'BY',
 'A',
 'MARGIN',
 'PLUS',
 'SOR',
 'PER',
 'ANNUM.AS',
 'AT',
 'THE',
 'REPORTING',
 'DATE,',
 'THE',
 'GROUP',
 'HAS',
 'IN',
 'PLACE',
 'UNSECURED',
 'BORROWINGS',
 'COMPRISING:(A)',
 'TERM',
 'LOANS',
 'AND',
 'REVOLVING',
 'CREDIT',
 'FACILITIES(I)',
 '4-YEAR',
 '$150',
 'MILLION',
 'LOAN',
 'FACILITY',
 'FROM',
 'CIMB',
 'MATURING',
 'IN',
 'JUNE',
 '2019',
 '(FITLF1FL)',
 'CONSISTING',
 'OF:',
 'Ł',
 'FACILITY',
 'A:',
 '$100',
 'MILLION',
 'TERM',
 'LOAN',
 'FACILITY',
 'AT',
 'A',
 '˜XED',
 'RATE',
 'OF',
 '3.6%',
 'PER',
 'ANNUM',
 'FOR',
 '3.5',
 'YEARS',
 'FROM',
 'THE',
 'DATE',
 'OF',
 'LOAN',
 'DRAWN',
 'DOWN;',
 'AND',
 'Ł',
 'FACILITY',
 'B:',
 '$50',
 'MILLION',
 'REVOLVING',
 'CREDIT',
 'FACILITY',
 'AT',
 'AN',
 'INTEREST',
 'RATE',
 'OF',
 'MARGIN',
 'PLUS',
 'SWAP',
 'OFFER',
 'RATE.',
 '(II)

page : 229 /230
..............

page : 76 /230page 77
The words was found 5 times in total
table found
search col
search col
unable to find index
page : 84 /230page 85
The words was found 7 times in total
table found
search col
search col
have name
index : 30
start cleaning


,"FINANCIALS SUCH AS TALENT MANAGEMENT, OCCUPANCY RATE AND RISK",THE CEO ON A NAMED BASIS WITH A BREAKDOWN (IN PERCENTAGE
0,MANAGEMENT AND CONTROL PROCESSES. LINKING STI ...,OR DOLLAR TERMS) OF EACH DIRECTOR’S AND THE CE...
1,MANAGER CORPORATE SCORECARD HELPS TO ACHIEVE S...,"EARNED THROUGH BASE/FIXED SALARY, VARIABLE OR ..."
2,OF ESR-REIT WHICH ARE ALIGNED TO THE INTERESTS...,"INCOME/BONUSES, BENEFITS-IN-KIND, STOCK OPTION..."
3,,"BASED INCENTIVES AND AWARDS, AND OTHER LONG-TE..."
4,GIVEN THE GROWTH PROFILE AND FOOTPRINT OF ESR-...,(II) THE REMUNERATION OF AT LEAST THE TOP FIVE...
5,TRANSACTIONS WILL LIKELY BE A KEY FEATURE OF E...,OFFICERS (WHO ARE NEITHER DIRECTORS NOR THE CE...
6,PLAN. STRATEGIC TRANSACTIONS E.G. MERGERS & AC...,"S$250,000, WITH A BREAKDOWN (IN PERCENTAGE OR ..."
7,OTHER TRANSFORMATIONAL DEALS CAN BE LONG-GESTA...,OF EACH KEY EXECUTIVE OFFICER’S REMUNERATION E...
8,AND RESOURCE CONSUMING WHICH REQUIRES SPECIAL ...,"BASE/FIXED SALARY, VARIABLE OR PERFORMANCE-REL..."
9,CARE AND OFTEN OUT OF OFFICE WORKING HOURS SAC...,"BONUSES, BENEFITS-IN-KIND, STOCK OPTIONS GRANT..."


./missed_out_pdf_files/uncleaned csv/ESR FUNDS MANAGEMENT (S) LIMITED shares.csv
True
file is closed
['ELEC & ELTEK INTERNATIONAL COMPANY LIMITED.']


save errors in a list

In [17]:
print(len(unable_to_find_list))
if not os.path.isdir('errors'):
    os.makedirs('errors')
df = pd.DataFrame()
df["Unable to crawl from pdf companies"] = unable_to_find_list
# df.to_csv("./errors/crawl_from_pdf_fail.csv", index = False)

195


<div class="alert alert-block alert-info">
<b>Tried to Do:</b> 
Tried to write a script to move the unextracted file,but due to pdf extraction not being able to achieve 100% accuracy as each pdf is different,the script could cause more harm than good.</div>